### Reduce ERA5 and ERA-I Data

This will simplify the data in the regridded ERA-I and ERA5 files to look at only 1000mb temps in order to calculate a lat/lon mean and store them as new data files in atm561/final_proj/rd_data 

In [8]:
import xarray as xr
import pandas as pd
import numpy as np

In [9]:
ct_ds = xr.open_dataset('/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERAI_ct_rgrd.nc')
stdvt_ds = xr.open_dataset('/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERAI_stdvt_rgrd.nc')
jan_ds = xr.open_dataset('/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERA5_jant_rgrd.nc')
feb_ds = xr.open_dataset('/home11/grad/2021/gd673524/research/gdines/data/regrid_data/ERA5_febt_rgrd.nc')

In [10]:
### Vertical level specificaton
vlevel = 1000
levelStr = str(vlevel)

#Hour Specification
Hour = 0
hourStr = str(Hour)

#south_boxtest = [[40,256], [40,271], [30,271], [30,256],[40,256]]
#north_box3 = [[50,252], [50,273], [40,273], [40,252],[50,252]]
#Box coordinates:
lat_min = 30
lat_mid = 40
lat_max = 50
lon_min = 252
lon_mins = 256
lon_max = 273
lon_maxs = 271

In [11]:
#Reducing the Climatology dataset For north box
ct_rd = ct_ds.sel(lev=vlevel, hour=hourStr, lat=slice(lat_mid,lat_max), lon=slice(lon_min,lon_max))
ct_rdn = ct_rd.reindex(lat=list(reversed(ct_rd.lat)))

stdvt_rd = stdvt_ds.sel(lev=vlevel, hour=hourStr, lat=slice(lat_mid,lat_max), lon=slice(lon_min,lon_max)) #<-- this is climo at 1000mb for on
stdvt_rdn = stdvt_rd.reindex(lat=list(reversed(stdvt_rd.lat)))

#Reducing the Climatology dataset For south box
ct_rd = ct_ds.sel(lev=vlevel, hour=hourStr, lat=slice(lat_min,lat_mid), lon=slice(lon_mins,lon_maxs))
ct_rds = ct_rd.reindex(lat=list(reversed(ct_rd.lat)))

stdvt_rd = stdvt_ds.sel(lev=vlevel, hour=hourStr, lat=slice(lat_min,lat_mid), lon=slice(lon_mins,lon_maxs)) #<-- this is climo at 1000mb for on
stdvt_rds = stdvt_rd.reindex(lat=list(reversed(stdvt_rd.lat)))

In [12]:
ct_rdn

<xarray.Dataset>
Dimensions:  (lat: 7, time: 366, lon: 15)
Coordinates:
  * lat      (lat) float64 49.5 48.0 46.5 45.0 43.5 42.0 40.5
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
    hour     float64 0.0
    lev      int32 1000
  * lon      (lon) float64 252.0 253.5 255.0 256.5 ... 268.5 270.0 271.5 273.0
Data variables:
    t        (time, lat, lon) float32 ...
Attributes:
    regrid_method:  bilinear

In [13]:
#ctn_llmn = ct_rdn.mean(dim=['lat','lon'])
#stdvtn_llmn = stdvt_rdn.mean(dim=['lat', 'lon'])

#cts_llmn = ct_rds.mean(dim=['lat','lon'])
#stdvts_llmn = stdvt_rds.mean(dim=['lat', 'lon'])

In [14]:
#Reducing the ERA5 dataset For north box
jant_rdn = jan_ds.sel(level=vlevel, time=jan_ds.t.time.dt.hour == 0, lat=slice(lat_mid,lat_max), 
                      lon=slice(lon_min,lon_max))
jant_rdn = jant_rdn.reindex(lat=list(reversed(jant_rdn.lat)))

febt_rdn = feb_ds.sel(level=vlevel, time=feb_ds.t.time.dt.hour == 0, lat=slice(lat_mid,lat_max), 
                      lon=slice(lon_min,lon_max))
febt_rdn = febt_rdn.reindex(lat=list(reversed(febt_rdn.lat)))

#Reducing the ERA5 dataset For south box
jant_rds = jan_ds.sel(level=vlevel, time=jan_ds.t.time.dt.hour == 0, lat=slice(lat_min,lat_mid), 
                      lon=slice(lon_mins,lon_maxs))
jant_rds = jant_rds.reindex(lat=list(reversed(jant_rds.lat)))

febt_rds = feb_ds.sel(level=vlevel, time=feb_ds.t.time.dt.hour == 0, lat=slice(lat_min,lat_mid), 
                      lon=slice(lon_mins,lon_maxs))
febt_rds = febt_rds.reindex(lat=list(reversed(febt_rds.lat)))

In [15]:
north_era5_rd = xr.concat([jant_rdn,febt_rdn], dim='time')
south_era5_rd = xr.concat([jant_rds,febt_rds], dim='time')

In [10]:
#jantn_llmn = jant_rdn.mean(dim=['lat','lon'])
#febtn_llmn = febt_rdn.mean(dim=['lat', 'lon'])

#jants_llmn = jant_rds.mean(dim=['lat','lon'])
#febts_llmn = febt_rds.mean(dim=['lat', 'lon'])

In [11]:
#north_era5_llmn = xr.concat([jantn_llmn,febtn_llmn], dim='time')
#south_era5_llmn = xr.concat([jants_llmn,febts_llmn], dim='time')

In [16]:
ct_rdn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/ct_rdn.nc', mode='w')
ct_rds.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/ct_rds.nc', mode='w')

stdvt_rdn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/north_erai_stdvt_rd.nc', mode='w')
stdvt_rds.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/south_erai_stdvt_rd.nc', mode='w')

north_era5_rd.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/north_era5_rd.nc', mode='w')
south_era5_rd.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/south_era5_rd.nc', mode='w')

In [12]:
#ctn_llmn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/north_erai_t_llmn.nc', mode='w')
#cts_llmn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/south_erai_t_llmn.nc', mode='w')

#stdvtn_llmn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/north_erai_stdvt_llmn.nc', mode='w')
#stdvts_llmn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/south_erai_stdvt_llmn.nc', mode='w')

#north_era5_llmn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/north_era5_llmn.nc', mode='w')
#south_era5_llmn.to_netcdf(path='/home11/grad/2021/gd673524/atm561/final_proj/rd_data/south_era5_llmn.nc', mode='w')